In [8]:
#AIMS_essay_codes
'''Created by Arinze Lawrence Folarin
Date: 25th August, 2020
Project: An Analysis of Bandit Algorithms on Malaria Control.
Title: AIMS master essay codes
'''

'Created by Arinze Lawrence Folarin\nDate: 25th August, 2020\nProject: An Analysis of Bandit Algorithms on Malaria Control.\nTitle: AIMS master essay codes\n'

## Implementation and perfomance of Bandit Algorithms on Gaussian Environment

In [13]:
# A Modified code of: 
#2016-2018 Shangtong Zhang(zhangshangtong.cpp@gmail.com)             
# 2016 Tian Jun(tianjun.cpp@gmail.com)                                
# 2016 Artem Oboturov(oboturov@gmail.com)           
# 2016 Kenta Shimada(hyperkentakun@gmail.com)  

In [14]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from tqdm import trange
%matplotlib inline
matplotlib.use('TkAgg')

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from tqdm import trange
%matplotlib inline
#matplotlib.use('Agg')



class Bandit:
    # @k_arm: # of arms
    # @epsilon: probability for exploration in epsilon-greedy algorithm
    # @initial: initial estimation for each action
    # @step_size: constant step size for updating estimations
    # @sample_averages: if True, use sample averages to update estimations instead of constant step size
    # @UCB_param: if not None, use UCB algorithm to select action
    # @gradient: if True, use gradient based bandit algorithm
    # @gradient_baseline: if True, use average reward as baseline for gradient based bandit algorithm
    # @decay1: if True,compute the value of epsilon using function 3
    # @decay: if True,compute the value of epsilon using function 2
    # @decay2: if True,compute the value of epsilon using function 1
    def __init__(self, k_arm=10, epsilon=0., initial=0., step_size=0.1, sample_averages=False, UCB_param=None,
                 gradient=False, decay=False,gradient_baseline=False, true_reward=0.,decay2=False,decay1=False,decay_eps=1.0,total_time=0):
        self.k = k_arm
        self.step_size = step_size
        self.sample_averages = sample_averages
        self.indices = np.arange(self.k)
        self.time = 0
        self.UCB_param = UCB_param
        self.decay = decay
        self.gradient = gradient
        self.gradient_baseline = gradient_baseline
        self.average_reward = 0
        self.true_reward = true_reward
        self.epsilon = epsilon
        self.decay2 = decay2
        self.decay_eps = decay_eps
        self.decay1 = decay1
        self.initial = initial
        self.total_time =total_time

    def reset(self):
        # real reward for each action
        self.q_true = np.random.randn(self.k) + self.true_reward

        # estimation for each action
        self.q_estimation = np.zeros(self.k) + self.initial  
        # # of chosen times for each action
        self.action_count = np.zeros(self.k)

        self.best_action = np.argmax(self.q_true) 

        self.time = 0

    # get an action for this bandit
    def act(self):
        #################################################
        # DECAYING E-GREEDY FUNCTION 2
        if self.decay:
            self.decay_eps = 1/np.log(self.time+0.00001)
            if np.random.rand()< self.decay_eps:
                return np.random.choice(self.indices)
        # DECAYING E-GREEDY FUNCTION 1
        if self.decay2:
            self.decay_eps = self.decay_eps*0.9
            if np.random.rand()<self.decay_eps:
                return np.random.choice(self.indices)
        ################################################
        # DECAYING E-GREEDY FUNCTION 3
        if self.decay1:
            num = (self.total_time-self.time)/self.total_time
            r = np.max(num,0)
            P_end = 0.1
            self.decay_eps = (self.decay_eps - P_end)* r + P_end
            if np.random.rand()<self.decay_eps:
                return np.random.choice(self.indices)
        ##########################################################
        # E-GREEDY
        if np.random.rand() < self.epsilon:
            #print(self.epsilon)
            return np.random.choice(self.indices)
        
        #################################################################
        # UCB METHOD
        if self.UCB_param is not None:
            UCB_estimation = self.q_estimation + \
                self.UCB_param * np.sqrt(np.log(self.time + 1) / (self.action_count + 1e-5))
            q_best = np.max(UCB_estimation) ## The UCB formular
            return np.random.choice(np.where(UCB_estimation == q_best)[0]) ## Returns the action
        # GRADIENT BANDIT
        if self.gradient:
            exp_est = np.exp(self.q_estimation) ## numerical preference exp^(H_t(a))
            self.action_prob = exp_est / np.sum(exp_est) ## Probability of taking action a at time t; pi_t(a)
            return np.random.choice(self.indices, p=self.action_prob) ## returns the action

        q_best = np.max(self.q_estimation) ## choose the highest estimated Q value
        return np.random.choice(np.where(self.q_estimation == q_best)[0]) # select the action with the corresponding estimated value

    # take an action, update estimation for this action
    def step(self, action):
        # generate the reward under N(real reward, 1)
        reward = np.random.randn() + self.q_true[action] ## R_t
        self.time += 1 ## t
        self.action_count[action] += 1 ## N_t(a) Number of times the action 'a' was selected.
        self.average_reward += (reward - self.average_reward) / self.time ## R_t bar

        if self.sample_averages:
            # update estimation using sample averages
            self.q_estimation[action] += (reward - self.q_estimation[action]) / self.action_count[action] ## Estimated value Q_t(a)
        elif self.gradient:
            one_hot = np.zeros(self.k)
            one_hot[action] = 1
            if self.gradient_baseline:
                baseline = self.average_reward
            else:
                baseline = 0
            self.q_estimation += self.step_size * (reward - baseline) * (one_hot - self.action_prob) # stochastic gradient ascent
        else:
            # update estimation with constant step size
            self.q_estimation[action] += self.step_size * (reward - self.q_estimation[action]) # Tracking Nonstationary Problem
        return reward

## 10-armed bandit simulation
def simulate(runs, time, bandits):
    rewards = np.zeros((len(bandits), runs, time))
    best_action_counts = np.zeros(rewards.shape)
    for i, bandit in enumerate(bandits):
        for r in trange(runs):
            bandit.reset()
            for t in range(time):
                action = bandit.act()
                reward = bandit.step(action)
                rewards[i, r, t] = reward
                if action == bandit.best_action:
                    best_action_counts[i, r, t] = 1
    mean_best_action_counts = best_action_counts.mean(axis=1)
    mean_rewards = rewards.mean(axis=1)
    return mean_best_action_counts, mean_rewards


## E-greedy plot

In [15]:
def figure1(runs=2000, time=1000):
    epsilons = [0, 0.1, 0.01] 
    bandits = [Bandit(epsilon=eps, sample_averages=True) for eps in epsilons]
    best_action_counts, rewards = simulate(runs, time, bandits)

    for eps, rewards in zip(epsilons, rewards):
        plt.plot(rewards, label='epsilon = %.02f' % (eps))
    plt.xlabel('steps')
    plt.ylabel('average reward')
    plt.legend()
    plt.show()
#figure1()

## Decaying e-greedy vs E-greedy plot

In [16]:
def figure2(runs=2000, time=1000):
    bandits = []
    bandits.append(Bandit(decay2=True, sample_averages=True)) 
    bandits.append(Bandit(decay_eps=0, sample_averages=True,decay=True))
    bandits.append(Bandit(decay1=True, sample_averages=True,total_time=time))
    bandits.append(Bandit(epsilon=0.1, sample_averages=True))
    best_action_counts,rewards = simulate(runs, time, bandits)

    labels = ["decay_eps=Function 1","decay_eps=Function 2","decay_eps=Function 3","eps=0.1"]
    for i in range(len(bandits)):
        plt.plot(rewards[i], label=labels[i])
        
    plt.xlabel('steps')
    plt.ylabel('average reward')
    plt.legend()
    plt.show()
#figure2()

## Optimistic initial values vs E-greedy plot

In [17]:
def figure3(runs=2000, time=1000):
    bandits = []
    bandits.append(Bandit(epsilon=0, initial=5, step_size=0.1))
    bandits.append(Bandit(epsilon=0.1, initial=0, step_size=0.1))
    best_action_counts, rewards = simulate(runs, time, bandits)
    
    plt.plot(rewards[0], label='epsilon = 0, q = 5')
    plt.plot(rewards[1], label='epsilon = 0.1, q = 0')

    plt.xlabel('Steps')
    plt.ylabel('Average rewards')
    plt.legend()
    plt.show()
#figure3()

## UCB plot

In [18]:
def figure4(runs=2000, time=1000):
    bandits = []
    bandits.append(Bandit(epsilon=0, UCB_param=0.5, sample_averages=True))
    bandits.append(Bandit(epsilon=0, UCB_param=1.0, sample_averages=True))
    bandits.append(Bandit(epsilon=0, UCB_param=2.0, sample_averages=True))

    best_action_counts, rewards= simulate(runs, time, bandits)

    plt.plot(rewards[0], label='UCB c = 0.5')
    plt.plot(rewards[1], label='UCB c = 1.0')
    plt.plot(rewards[2], label='UCB c = 2.0')
    plt.xlabel('Steps')
    plt.ylabel('Average reward')
    plt.legend()
    plt.show()
#figure4()

## UCB vs E-greedy plot

In [19]:
def figure4_1(runs=2000, time=1000):
    bandits = []
    bandits.append(Bandit(epsilon=0, UCB_param=2, sample_averages=True))
    bandits.append(Bandit(epsilon=0.1, sample_averages=True))
    best_action_counts, rewards = simulate(runs, time, bandits)

    plt.plot(rewards[0], label='UCB c = 2')
    plt.plot(rewards[1], label='epsilon greedy epsilon = 0.1')
    plt.xlabel('Steps')
    plt.ylabel('Average reward')
    plt.legend()
    plt.show()
#figure4_1()

## Gradient bandit plot (for alpha = 0.1 and 0.01)

In [20]:
def figure5(runs=2000, time=1000):
    bandits = []
    bandits.append(Bandit(gradient=True, step_size=0.1, gradient_baseline=True, true_reward=4))
    bandits.append(Bandit(gradient=True, step_size=0.1, gradient_baseline=False, true_reward=4))
    bandits.append(Bandit(gradient=True, step_size=0.01, gradient_baseline=True, true_reward=4))
    bandits.append(Bandit(gradient=True, step_size=0.01, gradient_baseline=False, true_reward=4))
    best_action_counts, _ = simulate(runs, time, bandits)
    labels = ['alpha = 0.1, with baseline',
              'alpha = 0.1, without baseline',
              'alpha = 0.01, with baseline',
              'alpha = 0.01, without baseline']

    for i in range(len(bandits)):
        plt.plot(best_action_counts[i], label=labels[i])
    plt.xlabel('Steps')
    plt.ylabel('% Optimal action')
    plt.legend()
    plt.show()    
#figure5()

## Gradient bandit plot 2 (for alpha = 0.1 and 0.4)

In [21]:
def figure5_1(runs=2000, time=1000):
    bandits = []
    bandits.append(Bandit(gradient=True, step_size=0.1, gradient_baseline=True, true_reward=4))
    bandits.append(Bandit(gradient=True, step_size=0.1, gradient_baseline=False, true_reward=4))
    bandits.append(Bandit(gradient=True, step_size=0.4, gradient_baseline=True, true_reward=4))
    bandits.append(Bandit(gradient=True, step_size=0.4, gradient_baseline=False, true_reward=4))
    best_action_counts, _ = simulate(runs, time, bandits)
    labels = ['alpha = 0.1, with baseline',
              'alpha = 0.1, without baseline',
              'alpha = 0.4, with baseline',
              'alpha = 0.4, without baseline']

    for i in range(len(bandits)):
        plt.plot(best_action_counts[i], label=labels[i])
    plt.xlabel('Steps')
    plt.ylabel('% Optimal action')
    plt.legend()
    plt.show()
#figure5_1()

## Gradient bandit vs E-greedy

In [22]:
def figure5_2(runs=2000, time=1000):
    bandits = []
    bandits.append(Bandit(epsilon=0.1, initial=0, step_size=0.1))
    bandits.append(Bandit(gradient=True, step_size=0.1, gradient_baseline=True, true_reward=4))
    bandits.append(Bandit(gradient=True, step_size=0.1, gradient_baseline=False, true_reward=4))
    bandits.append(Bandit(gradient=True, step_size=0.4, gradient_baseline=True, true_reward=4))
    bandits.append(Bandit(gradient=True, step_size=0.4, gradient_baseline=False, true_reward=4))
    best_action_counts, rewards = simulate(runs, time, bandits)
    labels = ["epsilon greedy epsilon = 0.1",'alpha = 0.1, with baseline',
              'alpha = 0.1, without baseline',
              'alpha = 0.4, with baseline',
              'alpha = 0.4, without baseline']

    for i in range(len(bandits)):
        plt.plot(best_action_counts[i], label=labels[i])
    plt.xlabel('Steps')
    plt.ylabel('% Optimal action')
    plt.legend()
    plt.show()
#figure5_2()